In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import pandas as pd

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# load dataset
filename = "/content/drive/My Drive/Latin Translation ML project/Dataset1.csv"
doc=load_doc(filename)

In [ ]:
dataset_original = pd.read_csv(filename,converters={i: str for i in range(0, 2)})
dataset_original

,great,māgnus
0,great,māgna
1,great,māgnum
2,his own,suus
3,her own,sua
4,its own,suum
...,...,...
18232,you,vōbīs
18233,you,vōs
18234,freeborn children,līberī
18235,swiftly,citō


In [ ]:
clean_pairs= dataset_original.to_numpy()

In [ ]:

# save clean pairs to file
save_clean_data(clean_pairs, 'english-latin.pkl')
# spot check
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-latin.pkl
[great] => [māgna]
[great] => [māgnum]
[his own] => [suus]
[her own] => [sua]
[its own] => [suum]
[other] => [alius]
[another] => [alius]
[other] => [alia]
[another] => [alia]
[other] => [aliud]
[another] => [aliud]
[much] => [multus]
[many] => [multus]
[much] => [multa]
[many] => [multa]
[much] => [multum]
[many] => [multum]
[your] => [tuus]
[your] => [tua]
[your] => [tuum]
[not any] => [nūllus]
[no one] => [nūllus]
[not any] => [nūlla]
[no one] => [nūlla]
[not any] => [nūllum]
[no one] => [nūllum]
[one] => [ūnus]
[one] => [ūna]
[one] => [ūnum]
[good] => [bonus]
[good] => [bona]
[good] => [bonum]
[whole] => [tōtus]
[entire] => [tōtus]
[whole] => [tōta]
[entire] => [tōta]
[whole] => [tōtum]
[entire] => [tōtum]
[first] => [prīmus]
[first] => [prīma]
[first] => [prīmum]
[situated above] => [superus]
[upper] => [superus]
[situated above] => [supera]
[upper] => [supera]
[situated above] => [superum]
[upper] => [superum]
[so great] => [tantus]
[so much] => [tantus]


In [ ]:
clean_pairs

array([['great', 'māgna'],
       ['great', 'māgnum'],
       ['his own', 'suus'],
       ...,
       ['freeborn children', 'līberī'],
       ['swiftly', 'citō'],
       ['from here', 'hinc']], dtype=object)

In [ ]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-latin.pkl')
# random shuffle
shuffle(raw_dataset)
# reduce dataset size
n_sentences = 13000
dataset = raw_dataset[:n_sentences]
# split into train/test
train, test = dataset[:11000], dataset[11000:]
# save
save_clean_data(dataset, 'english-latin-both.pkl')
save_clean_data(train, 'english-latin-train.pkl')
save_clean_data(test, 'english-latin-test.pkl')

Saved: english-latin-both.pkl
Saved: english-latin-train.pkl
Saved: english-latin-test.pkl


In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
 

Using TensorFlow backend.


In [ ]:

 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y
 
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model
 
# load datasets
dataset = load_clean_sentences('english-latin-both.pkl')
train = load_clean_sentences('english-latin-train.pkl')
test = load_clean_sentences('english-latin-test.pkl')
 
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
lat_tokenizer = create_tokenizer(dataset[:, 1])
lat_vocab_size = len(lat_tokenizer.word_index) + 1
lat_length = max_length(dataset[:, 1])
print('Latin Vocabulary Size: %d' % lat_vocab_size)
print('Latin Max Length: %d' % (lat_length))
 

English Vocabulary Size: 4363
English Max Length: 30
Latin Vocabulary Size: 7657
Latin Max Length: 19


In [ ]:
# prepare training data
trainX = encode_sequences(lat_tokenizer, lat_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(lat_tokenizer, lat_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

In [ ]:
# define model
model = define_model(lat_vocab_size, eng_vocab_size, lat_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize defined model
print(model.summary())
plot_model(model, to_file='Latinmodel2.png', show_shapes=True)
# fit model
filename = 'Latinmodel2.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=120, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 256)           1960192   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 30, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 4363)          1121291   
Total params: 4,132,107
Trainable params: 4,132,107
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11000 samples, validate on 2000 samples
Epoch 1/120
 - 151s - loss: 1.7040 - acc: 0.8819 - val_loss: 0.9970 - val_acc: 0.8937

Epoch 00001: val_loss improved from inf to 0.99696, saving model to Latinmodel2.h5
Epoch 2/120
 - 151s - loss: 0.8913 - acc: 0.8949 - val_loss: 0.8343 - val_acc: 0.8959

Epoch 00002: val_loss improved from 0.99696 to 0.83432, saving model to Latinmodel2.h5
Epoch 3/120
 - 150s - loss: 0.7935 - acc: 0.8978 - val_loss: 0.8005 - val_acc: 0.8966

Epoch 00003: val_loss improved from 0.83432 to 0.80047, saving model to Latinmodel2.h5
Epoch 4/120
 - 151s - loss: 0.7587 - acc: 0.9002 - val_loss: 0.7812 - val_acc: 0.8989

Epoch 00004: val_loss improved from 0.80047 to 0.78123, saving model to Latinmodel2.h5
Epoch 5/120
 - 150s - loss: 0.7258 - acc: 0.9018 - val_loss: 0.7657 - val_acc: 0.8990

Epoch 00005: val_loss improved from 0.78123 to 0.76566, saving model to Latinmodel2.h5
Epoch 6/120
 - 150s - loss: 0.7025 - acc: 0.9026 - val_loss: 0.7600 - val_acc: 0.8990

In [ ]:
!cp Latinmodel2.h5 drive/My\ Drive/

In [ ]:
import numpy as np
tk= Tokenizer()

In [ ]:

from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
model_path = '/content/Latinmodel2.h5'

In [ ]:
model = load_model(model_path)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 

In [ ]:
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))

In [ ]:
preds_text = []
for i in preds:
       temp = []
       for j in range(len(i)):
            t = word_for_id(i[j], eng_tokenizer)
            if j > 0: 
                if (t == word_for_id(i[j-1], eng_tokenizer)) or (t == None):
                     temp.append('')
                else:
                     temp.append(t)
            else:
                   if(t == None):
                          temp.append('')
                   else:
                          temp.append(t) 

       preds_text.append(' '.join(temp))

In [ ]:
pred_df = pd.DataFrame({'actual' : test[:,0], 'predicted' : preds_text})
pred_df.sample(60)


,actual,predicted
523,Painter unknown,unknown
1374,The following age will be amazed,it age be amazed
744,set forth,go
289,robe,of
1572,fire,of fire
379,which was to be done,which was to be done
464,this,this
1698,of sea,of plain
426,to forms,forms
1919,chances,o


In [ ]:
Input_latin = ['longa']
source = encode_sequences(lat_tokenizer,lat_length,Input_latin)
predict_sequence(model, eng_tokenizer, source)

'long'

In [ ]:

from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 
# load datasets
dataset = load_clean_sentences('english-latin-both.pkl')
train = load_clean_sentences('english-latin-train.pkl')
test = load_clean_sentences('english-latin-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
lat_tokenizer = create_tokenizer(dataset[:, 1])
lat_vocab_size = len(lat_tokenizer.word_index) + 1
lat_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(lat_tokenizer, lat_length, train[:, 1])
testX = encode_sequences(lat_tokenizer, lat_length, test[:, 1])
 
# load model
model = load_model('Latinmodel2.h5')
# test on some training sequences
print('train')
evaluate_model(model, lat_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, lat_tokenizer, testX, test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


train
src=[mīrātus sum], target=[wonder at], predicted=[wonder at]
src=[pugnis], target=[battle], predicted=[battle]
src=[cursus], target=[of advance], predicted=[o advance]
src=[tarda], target=[lingering], predicted=[sluggish]
src=[Elizabeth Regina/Eduardus Rex], target=[Queen Elizabeth/King Edward], predicted=[queen elizabeth king edward]
src=[ēdere], target=[state], predicted=[state]
src=[inimīca], target=[enemy], predicted=[enemy]
src=[hospes], target=[stranger], predicted=[guest]
src=[semper invicta], target=[always invincible], predicted=[always invincible]
src=[pietās], target=[devotion], predicted=[sense]
BLEU-1: 0.534008
BLEU-2: 0.356906
BLEU-3: 0.270156
BLEU-4: 0.153844
test
src=[Nulla regula sine exceptione], target=[There is no rule without exception], predicted=[no is without without without without without]
src=[clārus], target=[clear], predicted=[distinguished]
src=[quis leget haec?], target=[Who will read this?], predicted=[what who as as]
src=[Vita mutatur, non tollitu